In [10]:
import stable_baselines3 
from gym_2048.envs.game2048_env import Game2048Env
from random import randint
import matplotlib.pyplot as plt
import numpy as np
import copy
from collections import deque
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math

In [5]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.max_capacity = capacity
        self.buffer = deque([], maxlen=capacity)
        
    def push(self, item):
        self.buffer.append(item)

    def sample(self, sample_size):
        return random.sample(self.memory, sample_size)

In [7]:
class DQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 25)
        self.layer2 = nn.Linear(25, 25)
        self.layer3 = nn.Linear(25, n_actions)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [8]:
policy_network = DQN(25, 4)
target_network = DQN(25, 4)
target_network.load_state_dict(policy_network.state_dict())

c:\Users\navan\anaconda3\envs\AI\lib\site-packages\torch\nn\modules\linear.py:96: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))


<All keys matched successfully>

In [9]:
optimizer = optim.AdamW(policy_network.parameters(), lr=0.0004, amsgrad=True)
ReplayMemory = ReplayBuffer(1000)

In [11]:
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

steps_done = 0

def select_action(s):
    global steps_done
    prob = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)

    if prob > eps_threshold:
        with torch.no_grad():
            return policy_network(s).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[randint(0,3)]], device=device, dtype=torch.long)
    
